<a href="https://colab.research.google.com/github/KizuCode/argentina-fuel-analysis/blob/main/tp_4_limpieza_csv_combustibles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Código de limpieza y carga de datos del prototipo tecnológico:

<h1 align="center"><i><b>
Análisis de Consumo de Combustibles para la Toma de Decisiones Estratégicas de Expansión en Argentina
</b></i></h1>

---

## Recomendaciones
- Se sugiere tener al menos 2 GB de espacio disponible en la carpeta de Google Drive.
- Cambiar ruta en ```path = "/content/drive/MyDrive/Siglo 21/..."``` según la ubicación que se requiera.
- Cambiar a ```True``` en ```force_download = False``` para volver a descargar los archivos originales.
---

# Limpieza y normalización de datos

### Conexión con Google Drive y descarga de archivos originales

In [ ]:
# Montar Google Drive
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import unicodedata
import sqlite3
import shutil
import json
import os

# Ruta principal del proyecto
# Cambiar la dirección de la ruta según la ubicación del Google Drive
path = "/content/drive/MyDrive/Siglo 21/4º AÑO/Seminario Final en Lic. en Ciencia de Datos/4/"

# Carpeta donde se almacenará los archivos originales
# Se crea la carpeta en caso que no exista
csv_path = os.path.join(path, "csv")
os.makedirs(csv_path, exist_ok=True)

# Diccionario con los nombres de archivos a descargar y sus URLs de descarga
archivos = {
    "precios-en-surtidor-resolucion-3142016.csv": "http://datos.energia.gob.ar/dataset/1c181390-5045-475e-94dc-410429be4b17/resource/80ac25de-a44a-4445-9215-090cf55cfda5/download/precios-en-surtidor-resolucion-3142016.csv",
    "ventas-excluye-ventas-a-empresas-del-sector.csv": "http://datos.energia.gob.ar/dataset/5bdc436c-60d4-4c86-98ab-59834d047700/resource/f0e4e10a-e4b8-44e6-bd16-763a43742107/download/ventas-excluye-ventas-a-empresas-del-sector-.csv",
    "c2022_tp_est_c2.xlsx": "https://www.indec.gob.ar/ftp/cuadros/poblacion/c2022_tp_est_c2.xlsx"
}

# Opción para forzar descarga (True = si se desea descarga nuevamente, False = caso contrario)
force_download = False

# Descarga de archivos
for nombre, url in archivos.items():
    destino = os.path.join(csv_path, nombre)
    if force_download or not os.path.exists(destino):
        print(f"⬇️ Descargando {nombre} ...")
        !wget -O "{destino}" "{url}"
    else:
        print(f"✅ Ya existe: {destino}")

⬇️ Descargando precios-en-surtidor-resolucion-3142016.csv ...
--2025-11-12 12:31:08--  http://datos.energia.gob.ar/dataset/1c181390-5045-475e-94dc-410429be4b17/resource/80ac25de-a44a-4445-9215-090cf55cfda5/download/precios-en-surtidor-resolucion-3142016.csv
Resolving datos.energia.gob.ar (datos.energia.gob.ar)... 168.101.26.134
Connecting to datos.energia.gob.ar (datos.energia.gob.ar)|168.101.26.134|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9340914 (8.9M) [text/csv]
Saving to: ‘/content/drive/MyDrive/Siglo 21/4º AÑO/Seminario Final en Lic. en Ciencia de Datos/4/csv/precios-en-surtidor-resolucion-3142016.csv’

/content/drive/MyDr 100%[===================>]   8.91M  3.75MB/s    in 2.4s    

2025-11-12 12:31:12 (3.75 MB/s) - ‘/content/drive/MyDrive/Siglo 21/4º AÑO/Seminario Final en Lic. en Ciencia de Datos/4/csv/precios-en-surtidor-resolucion-3142016.csv’ saved [9340914/9340914]

⬇️ Descargando ventas-excluye-ventas-a-empresas-del-sector.csv ...
--2025-11

In [ ]:
# Carpeta donde se almacenará los archivos limpios
# Se crea la carpeta en caso que no exista
csv_limpio_path = os.path.join(path, "csv - limpio")
os.makedirs(csv_limpio_path, exist_ok=True)

# Función para leer csv y mostrar cantidad de filas cargadas
def leer_csv(csv, nombre):
  try:
    df = pd.read_csv(csv)
    print(f"✅ {nombre} cargado con {len(df)} filas")
    return df
  except Exception as e:
    print(f"⚠️ Error al cargar {nombre}: {e}")
    return None

# Carga de archivos originales en DataFrames
surtidor = leer_csv(os.path.join(csv_path, "precios-en-surtidor-resolucion-3142016.csv"), "Surtidor")
ventas   = leer_csv(os.path.join(csv_path, "ventas-excluye-ventas-a-empresas-del-sector.csv"), "Ventas")

✅ Surtidor cargado con 36961 filas
✅ Ventas cargado con 6310940 filas


In [ ]:
# Vista rápida de los primeros registros y estructura de los dataframes
print("Ventas:")
display(ventas.head(5))
print(ventas.info())
print("\nSurtidor:")
display(surtidor.head(5))
print(surtidor.info())

Ventas:


,anio,mes,empresa,tipodecomercializacion,subtipodecomercializacion,producto,unidad,provincia,cantidad,pais,indice_tiempo
0,2010,9,COMPAÑÍA GENERAL DE COMBUSTIBLES S.A.,Ventas,Otras Empresas,Aeronaftas(m3),(m3),Buenos Aires,0.0,no aplica,2010-09
1,2010,9,COMPAÑÍA GENERAL DE COMBUSTIBLES S.A.,Ventas,Otras Empresas,Aguarras(m3),(m3),Buenos Aires,0.0,no aplica,2010-09
2,2010,9,COMPAÑÍA GENERAL DE COMBUSTIBLES S.A.,Ventas,Otras Empresas,Asfaltos(Ton),(Ton),Buenos Aires,0.0,no aplica,2010-09
3,2010,9,COMPAÑÍA GENERAL DE COMBUSTIBLES S.A.,Ventas,Otras Empresas,Bases Lubricantes(m3),(m3),Buenos Aires,0.0,no aplica,2010-09
4,2010,9,COMPAÑÍA GENERAL DE COMBUSTIBLES S.A.,Ventas,Otras Empresas,Butano y Otros C4(Ton),(Ton),Buenos Aires,234.6,no aplica,2010-09


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6310940 entries, 0 to 6310939
Data columns (total 11 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   anio                       int64  
 1   mes                        int64  
 2   empresa                    object 
 3   tipodecomercializacion     object 
 4   subtipodecomercializacion  object 
 5   producto                   object 
 6   unidad                     object 
 7   provincia                  object 
 8   cantidad                   float64
 9   pais                       object 
 10  indice_tiempo              object 
dtypes: float64(1), int64(2), object(8)
memory usage: 529.6+ MB
None

Surtidor:


,indice_tiempo,idempresa,cuit,empresa,direccion,localidad,provincia,region,idproducto,producto,idtipohorario,tipohorario,precio,fecha_vigencia,idempresabandera,empresabandera,latitud,longitud,geojson
0,2025-10,1376,33-64337382-9,10 DE SETIEMBRE S.A.,Av. Mosconi 299,LOMAS DEL MIRADOR,BUENOS AIRES,PAMPEANA,19,Gas Oil Grado 2,2,Diurno,1525.0,2025-10-21 14:01:00,28,PUMA,-34.658476,-58.529443,"{""type"":""Point"",""coordinates"":[-58.529443,-34...."
1,2025-10,1376,33-64337382-9,10 DE SETIEMBRE S.A.,Av. Mosconi 299,LOMAS DEL MIRADOR,BUENOS AIRES,PAMPEANA,19,Gas Oil Grado 2,3,Nocturno,1525.0,2025-10-21 14:01:00,28,PUMA,-34.658476,-58.529443,"{""type"":""Point"",""coordinates"":[-58.529443,-34...."
2,2025-10,1376,33-64337382-9,10 DE SETIEMBRE S.A.,Av. Mosconi 299,LOMAS DEL MIRADOR,BUENOS AIRES,PAMPEANA,21,Gas Oil Grado 3,2,Diurno,1745.0,2025-10-21 14:02:00,28,PUMA,-34.658476,-58.529443,"{""type"":""Point"",""coordinates"":[-58.529443,-34...."
3,2025-10,1376,33-64337382-9,10 DE SETIEMBRE S.A.,Av. Mosconi 299,LOMAS DEL MIRADOR,BUENOS AIRES,PAMPEANA,21,Gas Oil Grado 3,3,Nocturno,1745.0,2025-10-21 14:02:00,28,PUMA,-34.658476,-58.529443,"{""type"":""Point"",""coordinates"":[-58.529443,-34...."
4,2025-05,1376,33-64337382-9,10 DE SETIEMBRE S.A.,Av. Mosconi 299,LOMAS DEL MIRADOR,BUENOS AIRES,PAMPEANA,6,GNC,2,Diurno,499.9,2025-05-30 12:05:00,28,PUMA,-34.658476,-58.529443,"{""type"":""Point"",""coordinates"":[-58.529443,-34...."


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36961 entries, 0 to 36960
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   indice_tiempo     36961 non-null  object 
 1   idempresa         36961 non-null  int64  
 2   cuit              36961 non-null  object 
 3   empresa           36961 non-null  object 
 4   direccion         36961 non-null  object 
 5   localidad         36961 non-null  object 
 6   provincia         36961 non-null  object 
 7   region            29082 non-null  object 
 8   idproducto        36961 non-null  int64  
 9   producto          36961 non-null  object 
 10  idtipohorario     36961 non-null  int64  
 11  tipohorario       36961 non-null  object 
 12  precio            36961 non-null  float64
 13  fecha_vigencia    36961 non-null  object 
 14  idempresabandera  36961 non-null  int64  
 15  empresabandera    36961 non-null  object 
 16  latitud           36877 non-null  float6

In [ ]:
# Nombre de meses en español
meses_esp = {
    1: "Enero", 2: "Febrero", 3: "Marzo", 4: "Abril",
    5: "Mayo", 6: "Junio", 7: "Julio", 8: "Agosto",
    9: "Septiembre", 10: "Octubre", 11: "Noviembre", 12: "Diciembre"
}

# Definir rango de fechas
start_fecha = ventas["indice_tiempo"].min()
end_fecha = surtidor["indice_tiempo"].max()
fechas = pd.date_range(start=str(start_fecha), end=str(end_fecha), freq="MS")

# Crear DataFrame de dimensión tiempo
dim_tiempo = pd.DataFrame({
    "indice_tiempo": fechas.strftime("%Y-%m"),
    "anio": fechas.year,
    "mes": fechas.month,
    "mes_nombre": fechas.month.map(meses_esp),
    "trimestre": fechas.quarter,
    "semestre": fechas.month.map(lambda x: 1 if x <= 6 else 2)
})

### Representación de los datos irregulares y Limpieza del Dataset de `ventas`

In [ ]:
# Filtrar filas con 'S/D' y 'Provincia'
filas_SD_Provincia = ventas[ventas["provincia"].isin(["S/D", "Provincia"])]

# Calcular totales y porcentaje
total_filas_ventas = ventas.shape[0]
total_filas_SD_Provincia = filas_SD_Provincia.shape[0]
pct_sd_provincia_sobre_total = total_filas_SD_Provincia / total_filas_ventas * 100

# Mostrar resultado
print(f"Se encontraron {total_filas_SD_Provincia} filas con valores 'S/D' y 'Provincia'.")
print(f"Estas filas representan un {round(pct_sd_provincia_sobre_total, 2)}% del total.")

Se encontraron 158016 filas con valores 'S/D' y 'Provincia'.
Estas filas representan un 2.5% del total.


In [ ]:
# Limpieza y normalización de la tabla ventas
ventas = ventas.drop("tipodecomercializacion", axis=1, errors="ignore")
ventas = ventas.rename(columns={"subtipodecomercializacion": "sector"})
ventas = ventas[ventas["pais"].isin(["ARGENTINA", "no aplica"])]
ventas = ventas[~ventas["provincia"].isin(["S/D", "Provincia"])]

# Normalizar provincias y sectores
ventas["provincia"] = ventas["provincia"].replace({
    "no aplica": "Estado Nacional"
})
ventas.loc[(ventas["provincia"] == "Estado Nacional") & (ventas["sector"] == "S/N"), "sector"] = "Estado Nacional"

# Limpiar nombres de productos y campos de texto
ventas["producto"] = ventas["producto"].str.replace(r"\s*\([^)]*\)$", "", regex=True).str.strip()
for col in ["empresa", "sector", "producto", "unidad", "provincia", "pais", "indice_tiempo"]:
    ventas[col] = ventas[col].apply(lambda x: x.strip() if isinstance(x, str) else x)

# Eliminar duplicados y quedarse con la primera aparición
ventas = ventas.drop_duplicates(keep="first")

### Limpieza del Dataset de `surtidor`

In [ ]:
# Limpiar campos de texto
for col in ["indice_tiempo", "empresa", "localidad", "provincia", "producto", "tipohorario", "empresabandera"]:
    surtidor[col] = surtidor[col].apply(lambda x: x.strip() if isinstance(x, str) else x)

# Dar nuevo formato de fecha con horas y minutos
surtidor["fecha_vigencia"] = pd.to_datetime(surtidor["fecha_vigencia"])
surtidor["fecha_vigencia"] = surtidor["fecha_vigencia"].dt.strftime("%Y-%m-%d %H:%M")

# Crear nueva columna para unir latitud y longitud, se tratan los casos en donde no aparezcan datos
surtidor["coordenadas"] = surtidor.apply(
    lambda row: f"{row["latitud"]},{row["longitud"]}"
    if pd.notnull(row["latitud"]) else "SIN DATOS",
    axis=1
)

# Limpieza de la tabla surtidor
surtidor = surtidor.drop(["geojson", "idempresa", "cuit", "direccion", "region", "idproducto", "idtipohorario", "idempresabandera"], axis=1, errors="ignore")

### Funciones de tratamiento de textos y Limpieza de nombres de provincias

In [ ]:
# Mapeo y funciones para normalizar nombres de provincias
mapa_provincias = {
    "CAPITAL FEDERAL": "CIUDAD AUTONOMA DE BUENOS AIRES",
    "Capital Federal": "CIUDAD AUTONOMA DE BUENOS AIRES",
    "CIUDAD AUTONOMA DE BUENOS": "CIUDAD AUTONOMA DE BUENOS AIRES",
    "CABA": "CIUDAD AUTONOMA DE BUENOS AIRES",
    "TIERRA DEL FUEGO, ANTARTIDA E ISLAS\nDEL ATLANTICO SUR (1)(3)": "TIERRA DEL FUEGO"
}

def quitar_acentos(texto):
    if isinstance(texto, str):
        texto = unicodedata.normalize("NFKD", texto)
        return "".join([c for c in texto if not unicodedata.combining(c)])
    return texto

def normalizar_provincia(p):
    if isinstance(p, str):
        p = quitar_acentos(p.strip().upper())
        return mapa_provincias.get(p, p)
    return p

In [ ]:
# Obtener y normalizar el nombre de las provincias obtenidas de ambos datasets (ventas y surtidor)
provincias_unicas = set(ventas["provincia"].dropna().apply(normalizar_provincia)) \
                    .union(set(surtidor["provincia"].dropna().apply(normalizar_provincia)))

dim_provincia = pd.DataFrame({
    "provincia_codigo": sorted(provincias_unicas)
})

# Diccionario con nombres legibles para mostrar resultados
provincias_dict = {
    "BUENOS AIRES": "Provincia de Buenos Aires",
    "CATAMARCA": "Catamarca",
    "CHACO": "Chaco",
    "CHUBUT": "Chubut",
    "CIUDAD AUTONOMA DE BUENOS AIRES": "Cdad. Autónoma de Buenos Aires",
    "CORDOBA": "Córdoba",
    "CORRIENTES": "Corrientes",
    "ENTRE RIOS": "Entre Ríos",
    "ESTADO NACIONAL": "Estado Nacional",
    "FORMOSA": "Formosa",
    "JUJUY": "Jujuy",
    "LA PAMPA": "La Pampa",
    "LA RIOJA": "La Rioja",
    "MENDOZA": "Mendoza",
    "MISIONES": "Misiones",
    "NEUQUEN": "Neuquén",
    "RIO NEGRO": "Río Negro",
    "SALTA": "Salta",
    "SAN JUAN": "San Juan",
    "SAN LUIS": "San Luis",
    "SANTA CRUZ": "Santa Cruz",
    "SANTA FE": "Santa Fe",
    "SANTIAGO DEL ESTERO": "Santiago del Estero",
    "TIERRA DEL FUEGO": "Tierra del Fuego",
    "TUCUMAN": "Tucumán"
}

# Diccionario con cada provincia y su región correspondiente
mapa_regiones = {
    "BUENOS AIRES": "PAMPEANA",
    "CATAMARCA": "NOA",
    "CHACO": "NEA",
    "CHUBUT": "PATAGONIA",
    "CIUDAD AUTONOMA DE BUENOS AIRES": "PAMPEANA",
    "CORDOBA": "CENTRO",
    "CORRIENTES": "NEA",
    "ENTRE RIOS": "CENTRO",
    "FORMOSA": "NEA",
    "JUJUY": "NOA",
    "LA PAMPA": "PATAGONIA",
    "LA RIOJA": "NOA",
    "MENDOZA": "CUYO",
    "MISIONES": "NEA",
    "NEUQUEN": "PATAGONIA",
    "RIO NEGRO": "PATAGONIA",
    "SALTA": "NOA",
    "SAN JUAN": "CUYO",
    "SAN LUIS": "CUYO",
    "SANTA CRUZ": "PATAGONIA",
    "SANTA FE": "CENTRO",
    "SANTIAGO DEL ESTERO": "NOA",
    "TIERRA DEL FUEGO": "PATAGONIA",
    "TUCUMAN": "NOA"
}

# Carga de datos a dim_provincia
dim_provincia["region"] = dim_provincia["provincia_codigo"].map(mapa_regiones)
dim_provincia["provincia_nombre"] = dim_provincia["provincia_codigo"].map(provincias_dict)

# Normalizar nombres de provincias en las tablas: ventas y surtidor
ventas["provincia"] = ventas["provincia"].apply(normalizar_provincia)
surtidor["provincia"] = surtidor["provincia"].apply(normalizar_provincia)

### Tratamiento del archivo `XLSX` y Limpieza del Dataset `poblacion`

In [ ]:
# Carga de archivo original, formato XLSX
censo_file = os.path.join(csv_path, "c2022_tp_est_c2.xlsx")

# Funciones auxiliares para normalizar texto y convertir series numéricas
def to_int_series(s: pd.Series) -> pd.Series:
    """
    Convierte una serie textual con separadores de miles (p. ej. '1,234,567') a int.
    """
    s = s.astype(str).str.replace(r'[,\.\s]', '', regex=True)
    return pd.to_numeric(s, errors='coerce').astype('Int64')

def quitar_tildes(texto: str) -> str:
    """
    Normaliza un string quitando tildes y pasándolo a minúsculas.
    """
    return ''.join(
        c for c in unicodedata.normalize('NFKD', texto.lower())
        if not unicodedata.combining(c)
    )

# Lectura del Excel con encabezado multinivel y normalización de columnas
# Leemos usando dos filas como encabezado (fila 4 y 5 en Excel → header=[3,4] en pandas)
df = pd.read_excel(censo_file, sheet_name="Cuadro 2", header=[3,4], dtype=str)

# Aplanar columnas (unir los dos niveles en un string limpio)
df.columns = [
    " ".join([str(c) for c in col if c and c != 'nan']).strip().lower()
    for col in df.columns.values
]

# Renombrar manualmente las primeras columnas
rename_map = {
    df.columns[0]: "codigo",
    df.columns[1]: "jurisdiccion",
    df.columns[2]: "superficie km2"
}
df = df.rename(columns=rename_map)

# Normalizar nombres de columnas (sin tildes)
df.columns = [quitar_tildes(c) for c in df.columns]

# Filtrado de provincias válidas
col_jur = "jurisdiccion"

provincias_validas = [
    "Ciudad Autónoma de Buenos Aires", "Buenos Aires", "Catamarca", "Chaco", "Chubut",
    "Córdoba", "Corrientes", "Entre Ríos", "Formosa", "Jujuy", "La Pampa", "La Rioja",
    "Mendoza", "Misiones", "Neuquén", "Río Negro", "Salta", "San Juan", "San Luis",
    "Santa Cruz", "Santa Fe", "Santiago del Estero",
    "Tierra del Fuego, Antártida e Islas\ndel Atlántico Sur (1)(3)", "Tucumán"
]

df = df[df[col_jur].isin(provincias_validas)].copy()

# Detección y limpieza de columnas de población
col_pob2010 = [c for c in df.columns if '2010' in c and 'poblacion' in c][0]
col_pob2022 = [c for c in df.columns if '2022' in c and 'poblacion' in c][0]

# Limpieza de población
df['poblacion 2010'] = to_int_series(df[col_pob2010])
df['poblacion 2022'] = to_int_series(df[col_pob2022])

# Transformación a formato largo con provincias y años
poblacion = pd.melt(
    df,
    id_vars=[col_jur],
    value_vars=['poblacion 2010', 'poblacion 2022'],
    var_name='año',
    value_name='poblacion'
)
poblacion['año'] = poblacion['año'].str.extract(r'(\d{4})').astype(int)
poblacion = poblacion.rename(columns={col_jur: 'provincia'})
poblacion['provincia'] = poblacion['provincia'].apply(normalizar_provincia)

In [ ]:
# Pivote para tener columnas 2010 y 2022 por provincia
pivot = poblacion.pivot(index="provincia", columns="año", values="poblacion").reset_index()

# Crear rango de años 2010–2025
anios = np.arange(2010, 2026)
rows = []

for _, row in pivot.iterrows():
    prov = row["provincia"]
    p2010 = row[2010]
    p2022 = row[2022]

    for anio in anios:
        # Interpolación/extrapolación lineal
        pred = p2010 + (p2022 - p2010) * ((anio - 2010) / (2022 - 2010))

        # Si el año es 2010 o 2022, usar el valor real
        if anio == 2010:
            pred = p2010
        elif anio == 2022:
            pred = p2022

        # Cargar resultados en la lista rows
        rows.append({
            "provincia": prov,
            "indice_tiempo": f"{anio}-01",
            "poblacion_pred": int(round(pred))
        })

# Nuevo DataFrame con las predicciones
poblacion = pd.DataFrame(rows)

### Revisión de duplicados y guardado de `CSV` limpios

In [ ]:
# Revisar duplicados
print('Filas duplicadas en ventas:', ventas.duplicated().sum())
print('Filas duplicadas en combustibles:', surtidor.duplicated().sum())

Filas duplicadas en ventas: 0
Filas duplicadas en combustibles: 0


In [ ]:
# Guardado de DataFrames en CSV
ventas_file = os.path.join(csv_limpio_path, 'ventas_limpio.csv')
ventas.to_csv(ventas_file, index=False)
print(f"✅ Archivo guardado: {ventas_file}")

surtidor_file = os.path.join(csv_limpio_path, 'surtidor_limpio.csv')
surtidor.to_csv(surtidor_file, index=False)
print(f"✅ Archivo guardado: {surtidor_file}")

poblacion_file = os.path.join(csv_limpio_path, "poblacion_provincias_2010_2022_estimaciones.csv")
poblacion.to_csv(poblacion_file, index=False, encoding="utf-8")
print(f"✅ Archivo guardado: {poblacion_file}")

provincia_file = os.path.join(csv_limpio_path, "provincia_region.csv")
dim_provincia.to_csv(provincia_file, index=False, encoding="utf-8")
print(f"✅ Archivo guardado: {provincia_file}")

tiempo_file = os.path.join(csv_limpio_path, "dim_tiempo.csv")
dim_tiempo.to_csv(tiempo_file, index=False)
print(f"✅ Archivo guardado: {tiempo_file}")

✅ Archivo guardado: /content/drive/MyDrive/Siglo 21/4º AÑO/Seminario Final en Lic. en Ciencia de Datos/4/csv - limpio/ventas_limpio.csv
✅ Archivo guardado: /content/drive/MyDrive/Siglo 21/4º AÑO/Seminario Final en Lic. en Ciencia de Datos/4/csv - limpio/surtidor_limpio.csv
✅ Archivo guardado: /content/drive/MyDrive/Siglo 21/4º AÑO/Seminario Final en Lic. en Ciencia de Datos/4/csv - limpio/poblacion_provincias_2010_2022_estimaciones.csv
✅ Archivo guardado: /content/drive/MyDrive/Siglo 21/4º AÑO/Seminario Final en Lic. en Ciencia de Datos/4/csv - limpio/provincia_region.csv
✅ Archivo guardado: /content/drive/MyDrive/Siglo 21/4º AÑO/Seminario Final en Lic. en Ciencia de Datos/4/csv - limpio/dim_tiempo.csv


# SQLite

### Creación de la base de datos, tablas, triggers y carga de tablas

In [ ]:
# Ruta del archivo de base de datos SQLite
db_file = os.path.join(path, 'dw_combustibles_arg.db')

In [ ]:
try:
    # --- Conexión y limpieza inicial ---
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()
    print(f"✅ Conectado a la base: {db_file}")

    # Desactivar claves foráneas y dropear tablas y triggers si existen
    cursor.execute('PRAGMA foreign_keys = OFF;')
    cursor.executescript('''
    DROP TABLE IF EXISTS ventas;
    DROP TABLE IF EXISTS surtidor;
    DROP TABLE IF EXISTS poblacion;
    DROP TABLE IF EXISTS dim_tiempo;
    DROP TABLE IF EXISTS dim_provincia;
    DROP TRIGGER IF EXISTS trg_ventas_insert;
    DROP TRIGGER IF EXISTS trg_ventas_update;
    DROP TRIGGER IF EXISTS trg_ventas_delete;
    DROP TRIGGER IF EXISTS trg_surtidor_insert;
    DROP TRIGGER IF EXISTS trg_surtidor_update;
    DROP TRIGGER IF EXISTS trg_surtidor_delete;
    DROP TRIGGER IF EXISTS trg_poblacion_insert;
    DROP TRIGGER IF EXISTS trg_poblacion_update;
    DROP TRIGGER IF EXISTS trg_poblacion_delete;
    ''')
    conn.commit()
    cursor.execute('PRAGMA foreign_keys = ON;')

    # --- Definición del esquema ---
    cursor.executescript('''
    CREATE TABLE IF NOT EXISTS dim_tiempo (
        indice_tiempo TEXT PRIMARY KEY,
        anio INTEGER,
        mes INTEGER,
        mes_nombre TEXT,
        trimestre INTEGER,
        semestre INTEGER
    );

    CREATE TABLE dim_provincia (
        id_provincia INTEGER PRIMARY KEY AUTOINCREMENT,
        provincia_codigo TEXT UNIQUE,
        provincia_nombre TEXT,
        region TEXT
    );

    CREATE TABLE IF NOT EXISTS poblacion (
        indice_tiempo TEXT,
        id_provincia INTEGER,
        poblacion INTEGER,
        FOREIGN KEY (indice_tiempo) REFERENCES dim_tiempo(indice_tiempo),
        FOREIGN KEY (id_provincia) REFERENCES dim_provincia(id_provincia)
    );

    CREATE TABLE IF NOT EXISTS ventas (
        indice_tiempo TEXT,
        id_provincia INTEGER,
        anio INTEGER,
        mes INTEGER,
        empresa TEXT,
        sector TEXT,
        producto TEXT,
        unidad TEXT,
        cantidad REAL,
        pais TEXT,
        FOREIGN KEY (indice_tiempo) REFERENCES dim_tiempo(indice_tiempo),
        FOREIGN KEY (id_provincia) REFERENCES dim_provincia(id_provincia)
    );

    CREATE TABLE IF NOT EXISTS surtidor (
        indice_tiempo TEXT,
        id_provincia INTEGER,
        empresa TEXT,
        localidad TEXT,
        producto TEXT,
        tipohorario TEXT,
        precio REAL,
        fecha_vigencia TEXT,
        empresabandera TEXT,
        latitud REAL,
        longitud REAL,
        coordenadas TEXT,
        FOREIGN KEY (indice_tiempo) REFERENCES dim_tiempo(indice_tiempo),
        FOREIGN KEY (id_provincia) REFERENCES dim_provincia(id_provincia)
    );

    CREATE TABLE IF NOT EXISTS bitacora_general (
        log_id INTEGER PRIMARY KEY AUTOINCREMENT,
        tabla_afectada TEXT,
        accion TEXT,
        fecha TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        detalle TEXT
    );
    ''')
    conn.commit()

    # --- Carga de CSVs limpios ---
    ventas_db = pd.read_csv(os.path.join(csv_limpio_path, 'ventas_limpio.csv'))
    surtidor_db = pd.read_csv(os.path.join(csv_limpio_path, 'surtidor_limpio.csv'))
    dim_tiempo_db = pd.read_csv(os.path.join(csv_limpio_path, 'dim_tiempo.csv'))
    poblacion_db = pd.read_csv(os.path.join(csv_limpio_path, 'poblacion_provincias_2010_2022_estimaciones.csv'))
    provincia_db = pd.read_csv(os.path.join(csv_limpio_path, 'provincia_region.csv'))

    # Insertar dimensiones base
    dim_tiempo_db.to_sql('dim_tiempo', conn, if_exists='append', index=False)
    provincia_db.to_sql('dim_provincia', conn, if_exists='append', index=False)

    # Recuperar mapping de provincias con id_provincia
    prov_map = pd.read_sql('SELECT id_provincia, provincia_codigo FROM dim_provincia', conn)

    # --- Normalización y carga de tablas de hechos ---
    # Ventas
    ventas_db['provincia_codigo'] = ventas_db['provincia'].apply(normalizar_provincia)
    ventas_db = ventas_db.merge(prov_map, on='provincia_codigo', how='left') \
                   .drop(columns=['provincia','provincia_codigo'])

    # Surtidor
    surtidor_db['provincia_codigo'] = surtidor_db['provincia'].apply(normalizar_provincia)
    surtidor_db = surtidor_db.merge(prov_map, on='provincia_codigo', how='left') \
                       .drop(columns=['provincia','provincia_codigo'])

    # Población
    poblacion_db['provincia_codigo'] = poblacion_db['provincia'].apply(normalizar_provincia)
    poblacion_db = poblacion_db.merge(prov_map, on='provincia_codigo', how='left') \
                               .drop(columns=['provincia','provincia_codigo'])
    poblacion_db = poblacion_db[['indice_tiempo','id_provincia','poblacion_pred']]
    poblacion_db = poblacion_db.rename(columns={"poblacion_pred": "poblacion"})

    # --- Insertar tablas ---
    ventas_db.to_sql('ventas', conn, if_exists='append', index=False)
    surtidor_db.to_sql('surtidor', conn, if_exists='append', index=False)
    poblacion_db.to_sql('poblacion', conn, if_exists='append', index=False)

    # --- Creación de los triggers ---
    cursor.executescript("""
    CREATE TRIGGER IF NOT EXISTS trg_ventas_insert
    AFTER INSERT ON ventas
    BEGIN
        INSERT INTO bitacora_general (tabla_afectada, accion, detalle)
        VALUES ('ventas', 'INSERT',
                'indice_tiempo=' || NEW.indice_tiempo ||
                ', id_provincia=' || NEW.id_provincia ||
                ', empresa=' || NEW.empresa);
    END;

    CREATE TRIGGER IF NOT EXISTS trg_ventas_update
    AFTER UPDATE ON ventas
    BEGIN
        INSERT INTO bitacora_general (tabla_afectada, accion, detalle)
        VALUES ('ventas', 'UPDATE',
                'indice_tiempo=' || NEW.indice_tiempo ||
                ', id_provincia=' || NEW.id_provincia ||
                ', empresa=' || NEW.empresa);
    END;

    CREATE TRIGGER IF NOT EXISTS trg_ventas_delete
    AFTER DELETE ON ventas
    BEGIN
        INSERT INTO bitacora_general (tabla_afectada, accion, detalle)
        VALUES ('ventas', 'DELETE',
                'indice_tiempo=' || OLD.indice_tiempo ||
                ', id_provincia=' || OLD.id_provincia ||
                ', empresa=' || OLD.empresa);
    END;

    CREATE TRIGGER IF NOT EXISTS trg_surtidor_insert
    AFTER INSERT ON surtidor
    BEGIN
        INSERT INTO bitacora_general (tabla_afectada, accion, detalle)
        VALUES ('surtidor', 'INSERT',
                'indice_tiempo=' || NEW.indice_tiempo ||
                ', id_provincia=' || NEW.id_provincia ||
                ', empresa=' || NEW.empresa ||
                ', producto=' || NEW.producto);
    END;

    CREATE TRIGGER IF NOT EXISTS trg_surtidor_update
    AFTER UPDATE ON surtidor
    BEGIN
        INSERT INTO bitacora_general (tabla_afectada, accion, detalle)
        VALUES ('surtidor', 'UPDATE',
                'indice_tiempo=' || NEW.indice_tiempo ||
                ', id_provincia=' || NEW.id_provincia ||
                ', empresa=' || NEW.empresa ||
                ', producto=' || NEW.producto);
    END;

    CREATE TRIGGER IF NOT EXISTS trg_surtidor_delete
    AFTER DELETE ON surtidor
    BEGIN
        INSERT INTO bitacora_general (tabla_afectada, accion, detalle)
        VALUES ('surtidor', 'DELETE',
                'indice_tiempo=' || OLD.indice_tiempo ||
                ', id_provincia=' || OLD.id_provincia ||
                ', empresa=' || OLD.empresa ||
                ', producto=' || OLD.producto);
    END;

    CREATE TRIGGER IF NOT EXISTS trg_poblacion_insert
    AFTER INSERT ON poblacion
    BEGIN
        INSERT INTO bitacora_general (tabla_afectada, accion, detalle)
        VALUES ('poblacion', 'INSERT',
                'indice_tiempo=' || NEW.indice_tiempo ||
                ', id_provincia=' || NEW.id_provincia ||
                ', poblacion=' || NEW.poblacion);
    END;

    CREATE TRIGGER IF NOT EXISTS trg_poblacion_update
    AFTER UPDATE ON poblacion
    BEGIN
        INSERT INTO bitacora_general (tabla_afectada, accion, detalle)
        VALUES ('poblacion', 'UPDATE',
                'indice_tiempo=' || NEW.indice_tiempo ||
                ', id_provincia=' || NEW.id_provincia ||
                ', poblacion=' || NEW.poblacion);
    END;

    CREATE TRIGGER IF NOT EXISTS trg_poblacion_delete
    AFTER DELETE ON poblacion
    BEGIN
        INSERT INTO bitacora_general (tabla_afectada, accion, detalle)
        VALUES ('poblacion', 'DELETE',
                'indice_tiempo=' || OLD.indice_tiempo ||
                ', id_provincia=' || OLD.id_provincia ||
                ', poblacion=' || OLD.poblacion);
    END;
    """)
    conn.commit()

    print("✅ Base de datos cargada con éxito")

except Exception as e:
    print("❌ Error al cargar la base de datos:", e)

finally:
    if 'conn' in locals():
        conn.close()
        print("🔒 Conexión cerrada")

✅ Conectado a la base: /content/drive/MyDrive/Siglo 21/4º AÑO/Seminario Final en Lic. en Ciencia de Datos/4/dw_combustibles_arg.db
✅ Base de datos cargada con éxito
🔒 Conexión cerrada


### Verificación de la integridad de la base de datos después de la carga masiva

In [ ]:
# Conectar a la base
conn = sqlite3.connect(db_file)
cursor = conn.cursor()
print(f"✅ Conectado a la base: {db_file}")

# Contar filas de cada tabla
q1 = """
SELECT 'dim_tiempo' AS tabla, COUNT(*) AS filas FROM dim_tiempo
UNION ALL
SELECT 'dim_provincia', COUNT(*) FROM dim_provincia
UNION ALL
SELECT 'ventas', COUNT(*) FROM ventas
UNION ALL
SELECT 'surtidor', COUNT(*) FROM surtidor
UNION ALL
SELECT 'poblacion', COUNT(*) FROM poblacion
UNION ALL
SELECT 'log_id', COUNT(*) FROM bitacora_general;
"""
print("👉 Cantidad de filas por tabla")
display(pd.read_sql(q1, conn))

# Examinar integridad de claves foráneas
q2 = """
-- Ventas: FK hacia dim_tiempo
SELECT 'ventas-tiempo' AS tabla, COUNT(*) AS huerfanos
FROM ventas v
LEFT JOIN dim_tiempo t ON v.indice_tiempo = t.indice_tiempo
WHERE t.indice_tiempo IS NULL

UNION ALL
-- Ventas: FK hacia dim_provincia
SELECT 'ventas-provincia', COUNT(*)
FROM ventas v
LEFT JOIN dim_provincia p ON v.id_provincia = p.id_provincia
WHERE p.id_provincia IS NULL

UNION ALL
-- Surtidor: FK hacia dim_tiempo
SELECT 'surtidor-tiempo', COUNT(*)
FROM surtidor s
LEFT JOIN dim_tiempo t ON s.indice_tiempo = t.indice_tiempo
WHERE t.indice_tiempo IS NULL

UNION ALL
-- Surtidor: FK hacia dim_provincia
SELECT 'surtidor-provincia', COUNT(*)
FROM surtidor s
LEFT JOIN dim_provincia p ON s.id_provincia = p.id_provincia
WHERE p.id_provincia IS NULL

UNION ALL
-- Poblacion: FK hacia dim_tiempo
SELECT 'poblacion-tiempo', COUNT(*)
FROM poblacion pb
LEFT JOIN dim_tiempo t ON pb.indice_tiempo = t.indice_tiempo
WHERE t.indice_tiempo IS NULL

UNION ALL
-- Poblacion: FK hacia dim_provincia
SELECT 'poblacion-provincia', COUNT(*)
FROM poblacion pb
LEFT JOIN dim_provincia p ON pb.id_provincia = p.id_provincia
WHERE p.id_provincia IS NULL;
"""
print("👉 Huérfanos en claves foráneas")
display(pd.read_sql(q2, conn))

# Ver estructura de tablas
q3 = "SELECT sql FROM sqlite_master WHERE type='table';"
print("👉 Definición de tablas en la base")
display(pd.read_sql(q3, conn))

conn.close()

✅ Conectado a la base: /content/drive/MyDrive/Siglo 21/4º AÑO/Seminario Final en Lic. en Ciencia de Datos/4/dw_combustibles_arg.db
👉 Cantidad de filas por tabla


,tabla,filas
0,dim_tiempo,191
1,dim_provincia,25
2,ventas,6093927
3,surtidor,36961
4,poblacion,384
5,log_id,0


👉 Huérfanos en claves foráneas


,tabla,huerfanos
0,ventas-tiempo,0
1,ventas-provincia,0
2,surtidor-tiempo,0
3,surtidor-provincia,0
4,poblacion-tiempo,0
5,poblacion-provincia,0


👉 Definición de tablas en la base


,sql
0,"CREATE TABLE sqlite_sequence(name,seq)"
1,CREATE TABLE dim_tiempo (\n indice_tiem...
2,CREATE TABLE dim_provincia (\n id_provi...
3,CREATE TABLE poblacion (\n indice_tiemp...
4,CREATE TABLE ventas (\n indice_tiempo T...
5,CREATE TABLE surtidor (\n indice_tiempo...
6,CREATE TABLE bitacora_general (\n log_i...


### Guardado de archivos `CSV` para uso en Looker Studio


In [ ]:
# Carpeta donde se almacenará los archivos para Looker Studio
# Se crea la carpeta en caso que no exista
sheet_path = os.path.join(path, "sheets - Looker Studio")
os.makedirs(sheet_path, exist_ok=True)

In [ ]:
# Conectar a la base
conn = sqlite3.connect(db_file)

"""
Para facilitar la carga de datos en Looker Studio se separa cada la tabla de ventas
en 4 partes, en donde cada una, se tiene una dimensión diferente.

Para las otras tablas se guardan según la consulta hecha en SQLite
"""

# Diccionario de consultas (Key = nombre del archivo; Valor = query)
querys = {
    "consumo_x_provincia_indice_tiempo.csv": """
    SELECT
        p.id_provincia,
        p.provincia_nombre,
        v.indice_tiempo,
        ROUND(SUM(v.cantidad), 2) AS consumo_total
    FROM ventas v
    JOIN dim_provincia p
    ON v.id_provincia = p.id_provincia
    GROUP BY p.provincia_nombre, v.indice_tiempo;
    """,
    "consumo_x_provincia_sector_indice_tiempo.csv": """
    SELECT
        p.id_provincia,
        p.provincia_nombre,
        v.indice_tiempo,
        v.anio,
        v.sector,
        ROUND(SUM(v.cantidad), 2) AS consumo_total
    FROM ventas v
    JOIN dim_provincia p
    ON v.id_provincia = p.id_provincia
    GROUP BY p.provincia_nombre, v.sector, v.indice_tiempo;
    """,
    "consumo_x_producto_sector_provinica.csv": """
    SELECT
        v.producto,
        v.sector,
        p.provincia_nombre,
        ROUND(SUM(v.cantidad), 2) AS consumo_total
    FROM ventas v
    JOIN dim_provincia p
    ON v.id_provincia = p.id_provincia
    GROUP BY v.producto, v.sector, p.provincia_nombre;
    """,
    "consumo_x_producto_sector_indice_tiempo.csv": """
    SELECT
        v.producto,
        v.sector,
        v.indice_tiempo,
        ROUND(SUM(v.cantidad), 2) AS consumo_total
    FROM ventas v
    JOIN dim_provincia p
    ON v.id_provincia = p.id_provincia
    GROUP BY v.producto, v.sector, v.indice_tiempo;
    """,
    "surtidor_dashboard.csv": """
    SELECT *
    FROM surtidor;
    """,
    "poblacion_dashboard.csv": """
    SELECT *
    FROM poblacion;
    """,
    "dim_tiempo_dashboard.csv": """
    SELECT *
    FROM dim_tiempo;
    """,
    "dim_provincia_dashboard.csv": """
    SELECT *
    FROM dim_provincia;
    """
}

# Se guardan los archivos CSV
for filename, sql in querys.items():
    try:
      df = pd.read_sql(sql, conn)
      file_path = os.path.join(sheet_path, filename)
      df.to_csv(file_path, index=False, encoding="utf-8")
      print(f"✅ Exportado: {file_path}")
    except Exception as e:
      print(f"❌ Error exportando {filename}: {e}")

conn.close()

✅ Exportado: /content/drive/MyDrive/Siglo 21/4º AÑO/Seminario Final en Lic. en Ciencia de Datos/4/sheets - Looker Studio/consumo_x_provincia_indice_tiempo.csv
✅ Exportado: /content/drive/MyDrive/Siglo 21/4º AÑO/Seminario Final en Lic. en Ciencia de Datos/4/sheets - Looker Studio/consumo_x_provincia_sector_indice_tiempo.csv
✅ Exportado: /content/drive/MyDrive/Siglo 21/4º AÑO/Seminario Final en Lic. en Ciencia de Datos/4/sheets - Looker Studio/consumo_x_producto_sector_provinica.csv
✅ Exportado: /content/drive/MyDrive/Siglo 21/4º AÑO/Seminario Final en Lic. en Ciencia de Datos/4/sheets - Looker Studio/consumo_x_producto_sector_indice_tiempo.csv
✅ Exportado: /content/drive/MyDrive/Siglo 21/4º AÑO/Seminario Final en Lic. en Ciencia de Datos/4/sheets - Looker Studio/surtidor_dashboard.csv
✅ Exportado: /content/drive/MyDrive/Siglo 21/4º AÑO/Seminario Final en Lic. en Ciencia de Datos/4/sheets - Looker Studio/poblacion_dashboard.csv
✅ Exportado: /content/drive/MyDrive/Siglo 21/4º AÑO/S

### Creación y Guardado de vistas y cálculo del coeficiente de Gini

In [ ]:
# Conectar a la base
conn = sqlite3.connect(db_file)
cursor = conn.cursor()
print(f"✅ Conectado a la base: {db_file}")

# --- Crear vistas ---
cursor.executescript('''
DROP VIEW IF EXISTS vw_participacion_provincia;
DROP VIEW IF EXISTS vw_crecimiento_interanual;
DROP VIEW IF EXISTS vw_estacionalidad;
DROP VIEW IF EXISTS vw_consumo_per_capita;

-- Participación provincial en el consumo nacional
CREATE VIEW vw_participacion_provincia AS
WITH consumo_provincia AS (
    SELECT
        p.provincia_nombre,
        v.anio,
        SUM(v.cantidad) AS consumo_total
    FROM ventas v
    JOIN dim_provincia p ON v.id_provincia = p.id_provincia
    GROUP BY p.provincia_nombre, v.anio
),
consumo_nacional AS (
    SELECT anio, SUM(consumo_total) AS total_pais
    FROM consumo_provincia
    GROUP BY anio
)
SELECT
    cp.provincia_nombre,
    printf('%s-01', cp.anio) AS anio,
    cp.consumo_total,
    ROUND(cp.consumo_total * 100.0 / cn.total_pais, 2) AS participacion_pct
FROM consumo_provincia cp
JOIN consumo_nacional cn ON cp.anio = cn.anio;

-- Crecimiento interanual
CREATE VIEW vw_crecimiento_interanual AS
WITH consumo_anual AS (
    SELECT
        p.provincia_nombre,
        v.anio,
        SUM(v.cantidad) AS consumo_total
    FROM ventas v
    JOIN dim_provincia p ON v.id_provincia = p.id_provincia
    GROUP BY p.provincia_nombre, v.anio
)
SELECT
    provincia_nombre,
    printf('%s-01', anio) AS anio,
    consumo_total,
    LAG(consumo_total) OVER (PARTITION BY provincia_nombre ORDER BY anio) AS consumo_anio_anterior,
    ROUND(
        ( (consumo_total - LAG(consumo_total) OVER (PARTITION BY provincia_nombre ORDER BY anio))
          * 100.0 / LAG(consumo_total) OVER (PARTITION BY provincia_nombre ORDER BY anio) ), 2
    ) AS crecimiento_pct
FROM consumo_anual;

-- Estacionalidad interna
CREATE VIEW vw_estacionalidad AS
WITH consumo_mensual AS (
    SELECT
        p.provincia_nombre,
        v.anio,
        v.mes,
        SUM(v.cantidad) AS consumo_mes
    FROM ventas v
    JOIN dim_provincia p ON v.id_provincia = p.id_provincia
    GROUP BY p.provincia_nombre, v.anio, v.mes
),
rango_anual AS (
    SELECT
        provincia_nombre,
        anio,
        MAX(consumo_mes) AS max_mes,
        MIN(consumo_mes) AS min_mes
    FROM consumo_mensual
    GROUP BY provincia_nombre, anio
)
SELECT
    provincia_nombre,
    printf('%s-01', anio) AS anio,
    max_mes,
    min_mes,
    ROUND(((max_mes - min_mes) * 100.0 / NULLIF(max_mes,0)),2) AS indice_estacionalidad_pct
FROM rango_anual;

-- Consumo per cápita (ventas + población)
CREATE VIEW vw_consumo_per_capita AS
SELECT
    p.provincia_nombre,
    printf('%s-01', v.anio) AS anio,
    SUM(v.cantidad) AS consumo_total,
    pop.poblacion,
    ROUND(SUM(v.cantidad) * 1.0 / pop.poblacion, 4) AS consumo_per_capita
FROM ventas v
JOIN dim_provincia p ON v.id_provincia = p.id_provincia
JOIN poblacion pop ON v.id_provincia = pop.id_provincia
JOIN dim_tiempo t ON pop.indice_tiempo = t.indice_tiempo AND v.anio = t.anio
GROUP BY p.provincia_nombre, printf('%s-01', v.anio), pop.poblacion;
''')

conn.commit()
conn.close()

✅ Conectado a la base: /content/drive/MyDrive/Siglo 21/4º AÑO/Seminario Final en Lic. en Ciencia de Datos/4/dw_combustibles_arg.db


In [ ]:
# Conectar a la base
conn = sqlite3.connect(db_file)
cursor = conn.cursor()
print(f"✅ Conectado a la base: {db_file}")

# Query para el calculo del coeficiente de Gini
query_calculo_gini = """
SELECT
    printf('%s-01', v.anio) AS anio,
    p.provincia_nombre,
    SUM(v.cantidad) AS consumo_total
FROM ventas v
JOIN dim_provincia p ON v.id_provincia = p.id_provincia
GROUP BY v.anio, p.provincia_nombre;
"""
año_x_total_consumo = pd.read_sql(query_calculo_gini, conn)
conn.close()

# Función del calculo de Gini
def gini(array):
    array = np.sort(array)              # Ordenar consumos
    n = len(array)
    cumvals = np.cumsum(array)          # Suma acumulada
    return (n + 1 - 2 * np.sum(cumvals) / cumvals[-1]) / n

# Armado del DataFrame
gini_por_anio = (
    año_x_total_consumo.groupby('anio')['consumo_total']
          .apply(lambda x: gini(x.values))
          .reset_index(name='coef_gini')
)

# Guardado de DataFrame en CSV
gini_file = os.path.join(sheet_path, 'gini_por_anio.csv')
gini_por_anio.to_csv(gini_file, index=False)
print(f"✅ Archivo guardado: {gini_file}")

✅ Conectado a la base: /content/drive/MyDrive/Siglo 21/4º AÑO/Seminario Final en Lic. en Ciencia de Datos/4/dw_combustibles_arg.db
✅ Archivo guardado: /content/drive/MyDrive/Siglo 21/4º AÑO/Seminario Final en Lic. en Ciencia de Datos/4/sheets - Looker Studio/gini_por_anio.csv


In [ ]:
# Conectar a la base
conn = sqlite3.connect(db_file)

# Lista de vistas a exportar
vistas = [
    "vw_participacion_provincia",
    "vw_crecimiento_interanual",
    "vw_estacionalidad",
    "vw_consumo_per_capita"
]

# Guardado de tablas en archivos CSV
for vista in vistas:
    try:
        df = pd.read_sql(f"SELECT * FROM {vista}", conn)
        file_path = os.path.join(sheet_path, f"{vista}.csv")
        df.to_csv(file_path, index=False, encoding='utf-8')
        print(f"✅ Vista {vista} exportada a {file_path}")
    except Exception as e:
        print(f"❌ Error exportando {vista}: {e}")

conn.close()

✅ Vista vw_participacion_provincia exportada a /content/drive/MyDrive/Siglo 21/4º AÑO/Seminario Final en Lic. en Ciencia de Datos/4/sheets - Looker Studio/vw_participacion_provincia.csv
✅ Vista vw_crecimiento_interanual exportada a /content/drive/MyDrive/Siglo 21/4º AÑO/Seminario Final en Lic. en Ciencia de Datos/4/sheets - Looker Studio/vw_crecimiento_interanual.csv
✅ Vista vw_estacionalidad exportada a /content/drive/MyDrive/Siglo 21/4º AÑO/Seminario Final en Lic. en Ciencia de Datos/4/sheets - Looker Studio/vw_estacionalidad.csv
✅ Vista vw_consumo_per_capita exportada a /content/drive/MyDrive/Siglo 21/4º AÑO/Seminario Final en Lic. en Ciencia de Datos/4/sheets - Looker Studio/vw_consumo_per_capita.csv


### Comprimir las carpetas `csv` y `csv - limpio` para gestión de almacenamiento

- Las carpetas borradas estarán en la Papelera para su eliminación manual

In [ ]:
# carpeta contenedora para los archivos comprimidos
zip_path = os.path.join(path, "archivos comprimidos - originales y limpios")
os.makedirs(zip_path, exist_ok=True)

# comprimir carpeta de originales
shutil.make_archive(
    os.path.join(zip_path, "original_data"),   # zip dentro de la carpeta contenedora
    'zip',
    csv_path                              # carpeta a comprimir
)

# comprimir carpeta de limpios
shutil.make_archive(
    os.path.join(zip_path, "clean_data"),
    'zip',
    csv_limpio_path
)

# eliminar carpetas originales, limpios y sheets
shutil.rmtree(csv_path, ignore_errors=True)
shutil.rmtree(csv_limpio_path, ignore_errors=True)

---
# Sector de consultas `SQLite` según necesidad

In [ ]:
# Conectar a la base
conn = sqlite3.connect(db_file)
cursor = conn.cursor()
print(f"✅ Conectado a la base: {db_file}")

# 1) Contar filas en cada tabla
q1 = """
SELECT
    printf("%s-01", v.anio) as anio,
    p.provincia_nombre,
    SUM(v.cantidad) AS consumo_total
FROM ventas v
JOIN dim_provincia p ON v.id_provincia = p.id_provincia
GROUP BY v.anio, p.provincia_nombre;
"""
display(pd.read_sql(q1, conn))

conn.close()

✅ Conectado a la base: /content/drive/MyDrive/Siglo 21/4º AÑO/Seminario Final en Lic. en Ciencia de Datos/4/dw_combustibles_arg.db


,anio,provincia_nombre,consumo_total
0,2010-01,Catamarca,1.608902e+05
1,2010-01,Cdad. Autónoma de Buenos Aires,3.966616e+06
2,2010-01,Chaco,4.605928e+05
3,2010-01,Chubut,5.143601e+05
4,2010-01,Corrientes,4.288897e+05
...,...,...,...
395,2025-01,Santa Cruz,2.750691e+05
396,2025-01,Santa Fe,2.051414e+06
397,2025-01,Santiago del Estero,2.977030e+05
398,2025-01,Tierra del Fuego,1.930226e+05
